In [ ]:
#default_exp lightcurve
%load_ext autoreload
from nbdev.showdoc import show_doc

# Light Curve
> Generate light curves from cells

In [ ]:
#export
import numpy as np
import pylab as plt
import pandas as pd
from utilities import  keyword_options

In [ ]:
#hide
from light_curves.config import LCconfig, FileConfiguration, PointSource
from light_curves.binner import get_cells

config = LCconfig()
files = FileConfiguration()
source = PointSource('Geminga')

if files.valid:
    print(f'Loading cells with source {source.name} ')
    cells = get_cells(config, files, source)
else:
    print('Not testing since no files.')
    


Loading cells with source Geminga 
Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	GTI MJD range: 54682.66-58698.08, good fraction 0.83 
Loading  132 months from Arrow dataset /home/burnett/data/dataset
....................................................................................................................................
	Selected 1313726 photons within 5 deg of  (195.13,4.27)
	Energies: 100.0-1000000 MeV
	Dates:    2008-08-04 15:46 - 2019-08-03 01:17
	MJD  :    54682.7          - 58698.1         
Load weights from file /mnt/c/users/thbur/OneDrive/fermi/weight_files/Geminga_weights.pkl
	Found: PSR J0633+1746 at (195.14, 4.27)
	Applyng weights: 240 / 1313726 photon pixels are outside weight region
	233109 weights set to NaN
Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	GTI MJD range: 54682.66-58698.08, good fraction 0.83 
Processing 12 S/C history (FT2) files
  applying cuts cos(theta) < 0.4,  z < 100
	fi

In [ ]:
#export
from light_curves.loglike import (GaussianRep, Gaussian2dRep, PoissonRep, PoissonRepTable)

class LightCurve(object):
    """ In the language of Kerr, manage a set of cells
    """
    defaults=(
        ('min_exp', 0.3, 'mimimum exposure factor'),
        ('rep',   'poisson', 'name of the likelihood representation: poisson, gauss, or gauss2d'),
        ('replist', 'gauss gauss2d poisson poisson_table'.split(), 'Possible reps'),
        ('rep_class', [GaussianRep, Gaussian2dRep, PoissonRep, PoissonRepTable], 'coresponding classes'),
        ('verbose', 0, 'verbosity'),
    )
    @keyword_options.decorate(defaults)
    def __init__(self, binned_weights, **kwargs):
        """Load binned data
        parameters:
            binned_weights : an iterable object that is a list of dicts; expect each
                to have following keys:
                    t, tw, fexp, w, S, B
            min_exp : minimum fractional exposure allowed
        """
        keyword_options.process(self,kwargs)
        global data
        self.data=data=binned_weights.data
        self.source_name = self.data.source_name
        self.verbose = getattr(data, 'verbose', self.verbose)

        # select the set of cells
        self.cells = [LogLike(ml) for ml in binned_weights if ml['fexp']>self.min_exp]
        if self.verbose>0:
            print(f'Loaded {len(self)} / {len(binned_weights)} cells with exposure >'\
                  f' {self.min_exp} for light curve analysis')

        # analyze using selected rep
        if self.rep not in self.replist:
            raise Exception(f'Unrecognized rep: "{self.rep}", must be one of {self.replist}')
        repcl = self.rep_class[self.replist.index(self.rep)]

        self.fit_df = self.fit(repcl)
#         try:
#             self.fit_df = self.fit(repcl)
#         except Exception as msg:
#             print(f'Fail fit: {msg} ') #'; stop here')
#             raise Exception(msg)

    def __repr__(self):
        return f'{self.__class__.__name__}: source "{self.source_name}" fit with {len(self.fit_df)} cells'

    def to_dict(self):
        return dict(source_name=self.source_name,
                    rep=self.rep,
                    edges = self.data.edges,
                    fit_dict = self.fit_df.to_dict('records'),
               )


    def write(self, filename):
        with open(filename, 'wb') as out:
            pickle.dump( self.to_dict(), out)

            if self.verbose>0:
                print(f'Wrote light curve for source "{self.source_name}" to {filename}')

    def __getitem__(self, i):
        return self.cells[i]

    def __len__(self):
        return len(self.cells)

    def fit(self, repcl):
        """Perform fits to all intervals with chosen log likelihood representataon
        """
        outd = dict()
        for i, cell in enumerate(self):
            try:
                outd[i]=repcl(cell).fit
            except Exception as msg:
                print(f'{self.__class__}, {repcl} fail for Index {i}, LogLike {cell}\n   {msg}')
                raise

        df = pd.DataFrame.from_dict(outd, orient='index', dtype=np.float32)
        if self.verbose>0:
            print(f'Fits using representation {self.rep}: {len(self)} intervals')
            if self.verbose>1:
                print(f'    columns: {list(df.columns)} ')
        return df

    @property
    def dataframe(self):
        """return the summary DataFrame
        """
        return self.fit_df

    def create_tables(self, npts=100, support=1e-6):
        """ create a set of tables representing the log-likelihoods of the cells
        parameters:
            npts : integer
                number of points
            support : float
                Integral of the Likelihood PDF to discard on each end

        returns
            arrays for points and log-likelihood values as (N, npts) arraays
        """
        assert self.rep.startswith('poisson'), 'Need a Poisson rep to calculate tables'
        df = self.dataframe
        dom = np.empty((len(df), npts), np.float32)
        cod = np.empty((len(df), npts), np.float32)

        def create_table(poiss):
            # make a table of evently-spaced points between limits
            a = poiss.cdfinv(support) if poiss.flux>0 else 0
            b = poiss.cdfcinv(support)
            x = np.linspace(a,b,npts).astype(np.float32)
            y = np.array(list(map(poiss, x))) .astype(np.float32)
            return x, y

        for i,poiss in enumerate(map( poisson.Poisson, df.poiss_pars.values)):
            dom[i], cod[i] = create_table(poiss)
        return dom, cod

    def flux_plot(self, ts_max=9, xerr=0.5, title=None, ax=None, **kwargs):
        """Make a plot of flux with according to the representation
        """
        kw=dict(yscale='linear',xlabel='MJD', ylabel='relative flux',)
        kw.update(**kwargs)
        df=self.fit_df
        if self.rep=='poisson':
            ts = df.ts
            limit = ts<ts_max
            bar = df.loc[~limit,:]
            lim = df.loc[limit,:]
        else:
            bar=df; lim=[]

        fig, ax = plt.subplots(figsize=(12,4)) if ax is None else (ax.figure, ax)\
            if ax is not None else (ax.figure,ax)

        # the points with error bars
        t = bar.t
        xerr = bar.tw/2
        y =  bar.flux.values
        if self.rep=='poisson':
            dy = [bar.errors.apply(lambda x: x[i]).clip(0,4) for i in range(2)]
        elif self.rep=='gauss' or self.rep=='gauss2d':
            dy = bar.sig_flux.clip(0,4)
        else: assert False
        ax.errorbar(x=t, y=y, xerr=xerr, yerr=dy, fmt=' ', color='silver')

        # now do the limits
        if len(lim)>0:
            t = lim.t
            xerr = lim.tw/2
            y = lim.limit.values
            yerr=0.2*(1 if kw['yscale']=='linear' else y)
            ax.errorbar(x=t, y=y, xerr=xerr,
                    yerr=yerr,  color='lightsalmon',
                    uplims=True, ls='', lw=2, capsize=4, capthick=0,
                    alpha=0.5)

        #ax.axhline(1., color='grey')
        ax.set(**kw)
        ax.set_title(title or f'{self.source_name}, rep {self.rep}')
        ax.grid(alpha=0.5)

    def fit_hists(self, title=None, fignum=None, **hist_kw):
        """### Plot distributions of rate, error, pull

        Source name:  "{source_name}"
        Bins fit with {self.rep} model.

        {fig.html}

        """
        source_name=self.source_name
        hkw = dict(log=True, histtype='stepfilled',lw=2, edgecolor='blue', facecolor='lightblue')
        hkw.update(hist_kw)

        df = self.fit_df
        fig, (ax1,ax2,ax3)= plt.subplots(1,3, figsize=(10,3.0), tight_layout=True, num=fignum)
        x = df.t
        y = df.flux
        if self.rep=='poisson': #mean of high, low
            yerr = df.errors.apply(lambda x: (x[0]+x[1])/2).clip(0,4)
        elif self.rep=='gauss' or self.rep=='gauss2d':
            yerr = df.sig_flux.clip(0,4)
        else: assert False

        def shist(ax, x,  xlim, nbins, label, xlog=False):
            def space(xlim, nbins=50):
                if xlog:
                    return np.logspace(np.log10(xlim[0]), np.log10(xlim[1]))
                return np.linspace(xlim[0], xlim[1], nbins+1)
            info = f'mean {x.mean():6.3f}\nstd  {x.std():6.3f}'
            ax.hist(x.clip(*xlim), bins=space(xlim, nbins), **hkw)
            ax.set(xlabel=label, xscale='log' if xlog else 'linear', ylim=(0.8,None))
            ax.text(0.62, 0.82, info, transform=ax.transAxes,fontdict=dict(size=10, family='monospace'))
            ax.grid(alpha=0.5)
            ax.xaxis.set_major_formatter(ticker.FuncFormatter(
                lambda val,pos: { 1.0:'1', 10.0:'10', 100.:'100', 5:'5', -5:'-5'}.get(val,'')))
            return ax

        shist(ax1, y, (0.25, 4), 25, 'relative flux', xlog=True).axvline(1, color='grey')
        shist(ax2, yerr*100, (1, 30), 25, 'sigma [%]', xlog=True)
        shist(ax3, (y-1)/yerr, (-6,6), 25,'pull').axvline(0,color='grey')
        return fig

    def mean_std(self):
        """ return weighted mean and rms"""
        df = self.fit_df
        frms = df.errors.apply(lambda err: 0.5*(err[0]+err[1]))
        fwts = 1/frms**2
        fmean = np.sum(df.flux*fwts)/np.sum(fwts)
        t = (df.flux-fmean)/frms
        return fmean, t.std()

Setup data to generate the light curve